In [82]:
# Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print("Delhi Air Quality Project - Starting Analysis")

Delhi Air Quality Project - Starting Analysis


In [81]:
# Loading AQI data
aqi_df = pd.read_csv('../data/raw/aqi.csv')
print("Data Loaded Successfully - Here are the columns:")
print(aqi_df.columns)


Data Loaded Successfully - Here are the columns:
Index(['date', 'state', 'area', 'number_of_monitoring_stations',
       'prominent_pollutants', 'aqi_value', 'air_quality_status', 'unit',
       'note'],
      dtype='object')


In [91]:
delhi_aqi = (
    aqi_df[
        aqi_df['state']
        .str.contains('delhi', case=False, na=False)
    ]
    .reset_index(drop=True)
)

# Preprocess date column and sort by date

delhi_aqi['date'] = pd.to_datetime(delhi_aqi['date'], format='%d-%m-%Y')
delhi_aqi = delhi_aqi.sort_values(by='date').reset_index(drop=True)

#handle missing values
print("missing values in delhi_aqi:", delhi_aqi.isnull().sum())

delhi_aqi['aqi_value'] = pd.to_numeric(delhi_aqi['aqi_value'], errors='coerce')
delhi_aqi = delhi_aqi.dropna(subset=['aqi_value'])

#check for duplicates
print("Duplicates in delhi_aqi:", delhi_aqi.duplicated().sum())

#removing unnecessary columns
delhi_aqi = delhi_aqi.drop(columns=['note'])



missing values in delhi_aqi: date                                0
state                               0
area                                0
number_of_monitoring_stations       0
prominent_pollutants                0
aqi_value                           0
air_quality_status                  0
unit                                0
note                             1125
dtype: int64
Duplicates in delhi_aqi: 0


In [94]:
print("\nAQI Data Summary:")


print(f"   AQI data: {delhi_aqi['date'].min().date()} to {delhi_aqi['date'].max().date()}")
print(f"   Total days: {len(aqi_df)}")
print(f"   Columns: {aqi_df.columns.tolist()}")
print("\nSample Data:")
print(aqi_df.head())
print(aqi_df.describe())

#Saving cleaned AQI data
delhi_aqi.to_csv('../data/processed/delhi_aqi_cleaned.csv', index=False)
print("Cleaned Delhi AQI data saved to '../data/processed/delhi_aqi_cleaned.csv'")


AQI Data Summary:
   AQI data: 2022-04-01 to 2025-04-30
   Total days: 235785
   Columns: ['date', 'state', 'area', 'number_of_monitoring_stations', 'prominent_pollutants', 'aqi_value', 'air_quality_status', 'unit', 'note']

Sample Data:
         date           state      area  number_of_monitoring_stations  \
0  30-04-2025     Maharashtra  Amravati                              2   
1  30-04-2025           Bihar    Purnia                              1   
2  30-04-2025  Madhya Pradesh     Katni                              1   
3  30-04-2025    Chhattisgarh   Tumidih                              1   
4  30-04-2025           Assam  Byrnihat                              1   

  prominent_pollutants  aqi_value air_quality_status  \
0                 PM10         78       Satisfactory   
1                   CO         56       Satisfactory   
2                   O3         98       Satisfactory   
3                 PM10        103           Moderate   
4                PM2.5         61   

In [95]:
# Loading weather data
weather_df = pd.read_csv('../data/raw/weather.csv')

weather_df.columns = [col.split(' (')[0] if '(' in col else col for col in weather_df.columns]

# Rename for simplicity
weather_df = weather_df.rename(columns={
    'time': 'date',
    'windspeed_10m_max': 'wind_speed_kmh',
    'precipitation_sum': 'precipitation_mm',
    'relative_humidity_2m_mean': 'humidity_percent'
})

#handle missing values
print("missing values in weather_df:", weather_df.isnull().sum())   

#check for duplicates
print("Duplicates in weather_df:", weather_df.duplicated().sum())

# Preprocess date column and sort by date
weather_df['date'] = pd.to_datetime(weather_df['date'])
weather_df = weather_df.sort_values('date').reset_index(drop=True)

# Convert wind speed from km/h to m/s (standard for pollution studies)
weather_df['wind_speed_ms'] = weather_df['wind_speed_kmh'] * 0.27778
weather_df = weather_df.drop(columns=['wind_speed_kmh'])

missing values in weather_df: date                   0
temperature_2m_max     0
temperature_2m_min     0
temperature_2m_mean    0
precipitation_mm       0
wind_speed_kmh         0
humidity_percent       0
dtype: int64
Duplicates in weather_df: 0


In [96]:
print("\nWeather Data Summary:")


print(f"   Weather data: {weather_df['date'].min().date()} to {weather_df['date'].max().date()}")
print(f"   Total days: {len(weather_df)}")
print(f"   Columns: {weather_df.columns.tolist()}")
print("\nSample Data:")
print(aqi_df.head())
print(aqi_df.describe())

#Saving cleaned weather data
weather_df.to_csv('../data/processed/delhi_weather_cleaned.csv', index=False)
print("Cleaned Delhi weather data saved to '../data/processed/delhi_weather_cleaned.csv'")


Weather Data Summary:
   Weather data: 2022-04-01 to 2025-04-30
   Total days: 1126
   Columns: ['date', 'temperature_2m_max', 'temperature_2m_min', 'temperature_2m_mean', 'precipitation_mm', 'humidity_percent', 'wind_speed_ms']

Sample Data:
         date           state      area  number_of_monitoring_stations  \
0  30-04-2025     Maharashtra  Amravati                              2   
1  30-04-2025           Bihar    Purnia                              1   
2  30-04-2025  Madhya Pradesh     Katni                              1   
3  30-04-2025    Chhattisgarh   Tumidih                              1   
4  30-04-2025           Assam  Byrnihat                              1   

  prominent_pollutants  aqi_value air_quality_status  \
0                 PM10         78       Satisfactory   
1                   CO         56       Satisfactory   
2                   O3         98       Satisfactory   
3                 PM10        103           Moderate   
4                PM2.5         

In [98]:
# 3. MERGE DATASETS
print("\n3. Merging AQI + Weather data...")
merged = pd.merge(delhi_aqi, weather_df, on='date', how='inner')

print(f"   Merged days: {len(merged)}")
print(f"   Common period: {merged['date'].min().date()} to {merged['date'].max().date()}")

# Check for missing data
missing_cols = merged.isnull().sum()
if missing_cols.sum() > 0:
    print(f"Missing values:\n{missing_cols[missing_cols > 0]}")
else:
    print("No missing values!")

# 4. SAVE MERGED DATASET
merged.to_csv('../data/processed/final_delhi_aqi_weather_merged_final.csv', index=False)
print("\n Merged dataset saved as 'final_delhi_aqi_weather_merged_final.csv'")



3. Merging AQI + Weather data...
   Merged days: 1125
   Common period: 2022-04-01 to 2025-04-30
No missing values!

 Merged dataset saved as 'final_delhi_aqi_weather_merged_final.csv'


In [104]:
print("DELHI AIR QUALITY + WEATHER ANALYSIS")
print("=" * 50)
print(f"Data Period: April 2022 - April 2025 ({1125} days)")
print("=" * 50)

# Load your merged data
merged = pd.read_csv('../data/processed/final_delhi_aqi_weather_merged_final.csv')
merged['date'] = pd.to_datetime(merged['date'])

print(f"Dataset Shape: {merged.shape}")
print(f"Date Range: {merged['date'].min().date()} to {merged['date'].max().date()}")
print(f"Total Days: {len(merged)}")

# Show available columns
print(f"\n Available Columns ({len(merged.columns)} total):")
for i, col in enumerate(merged.columns, 1):
    print(f"  {i:2}. {col}")

DELHI AIR QUALITY + WEATHER ANALYSIS
Data Period: April 2022 - April 2025 (1125 days)
Dataset Shape: (1125, 14)
Date Range: 2022-04-01 to 2025-04-30
Total Days: 1125

 Available Columns (14 total):
   1. date
   2. state
   3. area
   4. number_of_monitoring_stations
   5. prominent_pollutants
   6. aqi_value
   7. air_quality_status
   8. unit
   9. temperature_2m_max
  10. temperature_2m_min
  11. temperature_2m_mean
  12. precipitation_mm
  13. humidity_percent
  14. wind_speed_ms


In [105]:
# --------------------------------------------------------------------
# 1. BASIC STATISTICS
# --------------------------------------------------------------------
print("\n" + "="*50)
print("1. BASIC STATISTICS")
print("="*50)

# AQI Statistics
aqi_stats = merged['aqi_value'].describe()
print(f"\n AQI Statistics:")
print(f"   Average: {aqi_stats['mean']:.0f}")
print(f"   Minimum: {aqi_stats['min']:.0f} (Best day)")
print(f"   Maximum: {aqi_stats['max']:.0f} (Worst day)")
print(f"   Standard Deviation: {aqi_stats['std']:.0f}")

# Weather Statistics
print(f"\n Weather Statistics:")
if 'temperature_2m_mean' in merged.columns:
    print(f"   Avg Temperature: {merged['temperature_2m_mean'].mean():.1f}°C")
    print(f"   Range: {merged['temperature_2m_mean'].min():.1f}°C to {merged['temperature_2m_mean'].max():.1f}°C")

if 'wind_speed_ms' in merged.columns:
    print(f"   Avg Wind Speed: {merged['wind_speed_ms'].mean():.1f} m/s")
elif 'wind_speed_kmh' in merged.columns:
    merged['wind_speed_ms'] = merged['wind_speed_kmh'] * 0.27778
    print(f"   Avg Wind Speed: {merged['wind_speed_ms'].mean():.1f} m/s (converted from km/h)")

if 'precipitation_mm' in merged.columns:
    rainy_days = (merged['precipitation_mm'] > 0).sum()
    print(f"   Rainy Days: {rainy_days} ({rainy_days/len(merged)*100:.1f}% of days)")
    print(f"   Max Daily Rain: {merged['precipitation_mm'].max():.1f} mm")

if 'humidity_percent' in merged.columns:
    print(f"   Avg Humidity: {merged['humidity_percent'].mean():.0f}%")



1. BASIC STATISTICS

 AQI Statistics:
   Average: 206
   Minimum: 44 (Best day)
   Maximum: 494 (Worst day)
   Standard Deviation: 101

 Weather Statistics:
   Avg Temperature: 24.9°C
   Range: 8.2°C to 39.2°C
   Avg Wind Speed: 4.3 m/s
   Rainy Days: 383 (34.0% of days)
   Max Daily Rain: 116.0 mm
   Avg Humidity: 60%
